<a href="https://colab.research.google.com/github/truongkhng/Final_AI_Project/blob/main/Rice_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d muratkokludataset/rice-image-dataset
!unzip rice-image-dataset.zip

mkdir: cannot create directory ‘/root/.kaggle’: File exists
rice-image-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  rice-image-dataset.zip
replace Rice_Image_Dataset/Arborio/Arborio (1).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Rice_Image_Dataset/Arborio/Arborio (10).jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPool2D, BatchNormalization
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import cv2
from os import listdir
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [ ]:
def create_img_dataset(raw_folder,dataset_name):

    target_size = (250, 250)
    pictures = []
    labels = []
    #i = 0
    for folder in listdir(raw_folder):
      #print("Folder=",folder)
      for file in listdir(raw_folder + "/" + folder)[:2000]:
        #i+=1
        #print("File=", file)
        #print(i)
        pictures.append(cv2.resize(cv2.imread(raw_folder + "/" + folder + "/" + file)
                                              ,dsize = target_size))
        labels.append(folder)

    pictures = np.array(pictures)
    labels = np.array(labels)

    encoder = LabelBinarizer()
    labels = encoder.fit_transform(labels)

    with open (dataset_name, mode = 'wb') as file:
    # dump information to that file
      pickle.dump((pictures,labels), file)

    file.close()

def load_data(dataset_path):

    file = open(dataset_path, mode = 'rb')
    # dump information to that file
    (pictures, labels) = pickle.load (file)
    file.close()

    return pictures, labels

In [ ]:
raw_folder = "/content/Rice_Image_Dataset"

create_img_dataset(raw_folder= raw_folder,dataset_name= 'rice.txt')
dataset_path = "rice.txt"
x_data,y_data = load_data(dataset_path= 'rice.txt')

x_train, x_test, y_train, y_test = train_test_split( x_data, y_data, test_size=0.2, random_state=100)

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(8000, 250, 250, 3)
(8000, 5)
(2000, 250, 250, 3)
(2000, 5)


In [ ]:
x_test_original = x_test;
y_test_original = y_test;
x_label = ['Arborio','Barley','Basmati','Brown_rice','Ipsala','Japonica',
           'Jasmine','Jungwon','Karacadag','Sticky_rice']

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

In [ ]:
model = Sequential()
model.add (Conv2D(32,(5,5),activation='relu',kernel_initializer='normal',padding='same',
                  input_shape=(250,250,3)))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(32,(5,5),activation='relu',kernel_initializer='normal',padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(5,5),activation='relu',kernel_initializer='normal',padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(64,(5,5),activation='relu',kernel_initializer='normal',padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPool2D((2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu',kernel_initializer='normal'))
model.add(Dense(10,activation='softmax'))
model.summary()

In [ ]:
myImageGen = ImageDataGenerator(rotation_range=20,
                                width_shift_range = 0.1,
                                height_shift_range = 0.1,
                                horizontal_flip=True,
                                vertical_flip=True)

In [ ]:
checkpoint = ModelCheckpoint(filepath= "Rice-weights-{epoch:02d}-{val_accuracy:.4f}.h5",
                             monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
history = model.fit_generator(myImageGen.flow(x_train,y_train, batch_size = 32),epochs=100,
                              validation_data=myImageGen.flow(x_test,y_test, batch_size = 128),
                              verbose=1,callbacks=callbacks_list)

In [ ]:
score = model.evaluate(x_test, y_test, verbose = 0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
model_json = model.to_json()
with open('Rice_config.json', 'w') as json_file:
    json_file.write(model_json)

In [ ]:
# Show original Image
i = randint(0,len(y_test_original)-1)
print ("picture ",i)
print("actual:", y_test_original[i])
plt.imshow(x_test_original[i])

# Sample Predict
sample = np.array([x_test[i]])
predictions = np.argmax(model.predict(sample), axis=-1)
print("predictions:", predictions)
print("predictions label:", x_label[int(predictions)])

In [ ]:
# Graphs of accuracy
plt.figure(figsize=(8,5))
plt.grid(True)
#plt.gca().set_ylim(0,1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','validation'],loc='lower right')
plt.show()